# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

> NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [ ]:
#!pip install -qU ragas==0.2.10

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.7/175.7 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/1

In [ ]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

### NLTK Import

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

In [1]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nilayjhaveri/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/nilayjhaveri/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [3]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [4]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - which should hopefull be familiar at this point since it's our Loan Data use-case!

Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [21]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyMuPDFLoader


path = "data/"
loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyMuPDFLoader)
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [22]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [23]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [24]:
from ragas.testset.graph import Node, NodeType

### NOTICE: We're using a subset of the data for this example - this is to keep costs/time down.

for doc in docs[:20]:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 20, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [25]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/17 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/20 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/31 [00:00<?, ?it/s]

Property 'summary' already exists in node '7e5072'. Skipping!
Property 'summary' already exists in node '97fc74'. Skipping!
Property 'summary' already exists in node '591565'. Skipping!
Property 'summary' already exists in node 'db0b93'. Skipping!
Property 'summary' already exists in node '1f5fca'. Skipping!
Property 'summary' already exists in node '31c303'. Skipping!
Property 'summary' already exists in node 'a47535'. Skipping!
Property 'summary' already exists in node 'b65d1b'. Skipping!
Property 'summary' already exists in node 'c692ec'. Skipping!
Property 'summary' already exists in node '5854bb'. Skipping!
Property 'summary' already exists in node '195de4'. Skipping!
Property 'summary' already exists in node 'e87247'. Skipping!
Property 'summary' already exists in node 'b1f8cb'. Skipping!
Property 'summary' already exists in node '027815'. Skipping!


Applying CustomNodeFilter:   0%|          | 0/6 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/43 [00:00<?, ?it/s]

Property 'summary_embedding' already exists in node '1f5fca'. Skipping!
Property 'summary_embedding' already exists in node '7e5072'. Skipping!
Property 'summary_embedding' already exists in node 'a47535'. Skipping!
Property 'summary_embedding' already exists in node '5854bb'. Skipping!
Property 'summary_embedding' already exists in node 'db0b93'. Skipping!
Property 'summary_embedding' already exists in node '591565'. Skipping!
Property 'summary_embedding' already exists in node '97fc74'. Skipping!
Property 'summary_embedding' already exists in node '027815'. Skipping!
Property 'summary_embedding' already exists in node '31c303'. Skipping!
Property 'summary_embedding' already exists in node '195de4'. Skipping!
Property 'summary_embedding' already exists in node 'e87247'. Skipping!
Property 'summary_embedding' already exists in node 'b1f8cb'. Skipping!
Property 'summary_embedding' already exists in node 'b65d1b'. Skipping!
Property 'summary_embedding' already exists in node 'c692ec'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 40, relationships: 480)

We can save and load our knowledge graphs as follows.

In [26]:
kg.save("loan_data_kg.json")
loan_data_kg = KnowledgeGraph.load("loan_data_kg.json")
loan_data_kg

KnowledgeGraph(nodes: 40, relationships: 480)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [27]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=loan_data_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [28]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.


##### ✅ Answer:

**SingleHopSpecificQuerySynthesizer** - This generates straightforward, direct queries that can be answered with a single step or "hop" through the data. These are specific, focused questions that don't require complex reasoning or multiple data points to answer.

**MultiHopAbstractQuerySynthesizer** - This creates more complex queries that require multiple steps or "hops" to answer, but the questions are abstract in nature. These queries might need to connect different pieces of information or require higher-level reasoning to resolve.

**MultiHopSpecificQuerySynthesizer** - This generates complex queries that require multiple steps to answer, but unlike the abstract version, these queries are specific and concrete. They need multiple data points or reasoning steps but ask for precise, specific information.

Finally, we can use our `TestSetGenerator` to generate our testset!

In [29]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,What are academic years in school?,"[Chapter 1 Academic Years, Academic Calendars,...","Chapter 1 Academic Years, Academic Calendars, ...",single_hop_specifc_query_synthesizer
1,What is 34 CFR 668.3(a) about?,[Regulatory Citations Academic year minimums: ...,34 CFR 668.3(a) relates to the academic year m...,single_hop_specifc_query_synthesizer
2,What is Volume 8 in relation to clinical work ...,[Inclusion of Clinical Work in a Standard Term...,Inclusion of Clinical Work in a Standard Term:...,single_hop_specifc_query_synthesizer
3,What are Non-Term Characteristics in relation ...,[Non-Term Characteristics A program that measu...,A program that measures progress in credit-hou...,single_hop_specifc_query_synthesizer
4,What is the misspelled term related to student...,[both the credit or clock hours and the weeks ...,The term is 'Direct Loan'.,single_hop_specifc_query_synthesizer
5,Based on the guidance and regulations for Titl...,[<1-hop>\n\nboth the credit or clock hours and...,Disbursement timing in clock-hour or non-term ...,multi_hop_abstract_query_synthesizer
6,How do non-term characteristics like clock-hou...,[<1-hop>\n\nInclusion of Clinical Work in a St...,Non-term characteristics such as measuring pro...,multi_hop_abstract_query_synthesizer
7,How does the inclusion of clinical work outsid...,[<1-hop>\n\nInclusion of Clinical Work in a St...,The inclusion of clinical work conducted outsi...,multi_hop_abstract_query_synthesizer
8,How do Chapters 2 and 3 collectively inform th...,[<1-hop>\n\nInclusion of Clinical Work in a St...,Chapter 2 explains that clinical work conducte...,multi_hop_specific_query_synthesizer
9,Volume 2 and Volume 7 how relate to credit hou...,[<1-hop>\n\nboth the credit or clock hours and...,Volume 2 explains that the credit or clock hou...,multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [30]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs[:20], testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/17 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/20 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/31 [00:00<?, ?it/s]

Property 'summary' already exists in node '0a2726'. Skipping!
Property 'summary' already exists in node '8d0cc3'. Skipping!
Property 'summary' already exists in node '5d2301'. Skipping!
Property 'summary' already exists in node 'cf2ab0'. Skipping!
Property 'summary' already exists in node '6c5572'. Skipping!
Property 'summary' already exists in node 'd1e11d'. Skipping!
Property 'summary' already exists in node '45b047'. Skipping!
Property 'summary' already exists in node '8fb9f9'. Skipping!
Property 'summary' already exists in node '9687d0'. Skipping!
Property 'summary' already exists in node '2e212f'. Skipping!
Property 'summary' already exists in node '0643b6'. Skipping!
Property 'summary' already exists in node '9b88b2'. Skipping!
Property 'summary' already exists in node 'ae0bcf'. Skipping!
Property 'summary' already exists in node 'e79ced'. Skipping!


...

Property 'summary_embedding' already exists in node '0a2726'. Skipping!
Property 'summary_embedding' already exists in node '5d2301'. Skipping!
Property 'summary_embedding' already exists in node 'ae0bcf'. Skipping!
Property 'summary_embedding' already exists in node '9687d0'. Skipping!
Property 'summary_embedding' already exists in node 'cf2ab0'. Skipping!
Property 'summary_embedding' already exists in node '45b047'. Skipping!
Property 'summary_embedding' already exists in node '8d0cc3'. Skipping!
Property 'summary_embedding' already exists in node '0643b6'. Skipping!
Property 'summary_embedding' already exists in node '6c5572'. Skipping!
Property 'summary_embedding' already exists in node '2e212f'. Skipping!
Property 'summary_embedding' already exists in node 'e79ced'. Skipping!
Property 'summary_embedding' already exists in node '8fb9f9'. Skipping!
Property 'summary_embedding' already exists in node '9b88b2'. Skipping!
Property 'summary_embedding' already exists in node 'd1e11d'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [31]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,What is the role of the School Participation D...,"[Chapter 1 Academic Years, Academic Calendars,...",The context does not specify the exact role of...,single_hop_specifc_query_synthesizer
1,Could you please explain the significance of 3...,[Regulatory Citations Academic year minimums: ...,Regulatory Citations Academic year minimums: 3...,single_hop_specifc_query_synthesizer
2,What is Chaptr 3?,[Inclusion of Clinical Work in a Standard Term...,Inclusion of Clinical Work in a Standard Term ...,single_hop_specifc_query_synthesizer
3,FWS is it like a payment period or not and how...,[Non-Term Characteristics A program that measu...,The payment period is applicable to all Title ...,single_hop_specifc_query_synthesizer
4,How does completing additional hours or weeks ...,[<1-hop>\n\nboth the credit or clock hours and...,Completing additional weeks of instructional t...,multi_hop_abstract_query_synthesizer
5,Wht are the weeks of instrctional time reuired...,"[<1-hop>\n\nChapter 1 Academic Years, Academic...",The context states that for credit-hour progra...,multi_hop_abstract_query_synthesizer
6,How do credit-hour and clock-hour programs dif...,"[<1-hop>\n\nChapter 1 Academic Years, Academic...",Credit-hour programs must have an academic yea...,multi_hop_abstract_query_synthesizer
7,How does the inclusion of clinical work in sta...,[<1-hop>\n\nInclusion of Clinical Work in a St...,The inclusion of clinical work in standard ter...,multi_hop_abstract_query_synthesizer
8,How do Appendix A and Appendix B guide the dis...,[<1-hop>\n\nDisbursement Timing in Subscriptio...,Appendix B provides detailed guidance on disbu...,multi_hop_specific_query_synthesizer
9,How does Volume 8 explain the impact of accele...,[<1-hop>\n\nboth the credit or clock hours and...,Volume 8 details that in clock-hour or non-ter...,multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [33]:
from langsmith import Client

client = Client()

dataset_name = "Loan Synthetic Data"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Loan Synthetic Data"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [34]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [35]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [36]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [37]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [38]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Loan RAG"
)

In [39]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [40]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

In [41]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

Finally, we can set-up our RAG LCEL chain!

In [42]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [43]:
rag_chain.invoke({"question" : "What kinds of loans are available?"})

'The kinds of loans available mentioned in the context are:\n\n- Direct PLUS Loan (student Federal PLUS Loan)\n- Direct Subsidized Loan\n- Direct Unsubsidized Loan\n- Federal Stafford Loans (Subsidized and Unsubsidized) made under the FFEL Program before July 1, 2010\n- Federal SLS Loans\n- Federal PLUS Loans made under the FFEL Program before July 1, 2010\n\nAdditionally:\n\n- Direct Subsidized Loans are available only to undergraduate students.\n- Graduate or professional students are eligible for Direct Unsubsidized Loans and Direct PLUS Loans, but not Direct Subsidized Loans.\n\nTherefore, the available loan types include Direct Subsidized Loans, Direct Unsubsidized Loans, Direct PLUS Loans, and prior FFEL Program loans (Stafford, SLS, PLUS) made before July 1, 2010.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

In [44]:
eval_llm = ChatOpenAI(model="gpt-4.1")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [45]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

empathy_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "empathy": "Is this response empathetic? Does it make the user feel like they are being heard?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`:
- `labeled_helpfulness_evaluator`:
- `empathy_evaluator`:

##### ✅ Answer:

**qa_evaluator:**

Evaluates question-answer quality
Uses chain-of-thought reasoning to assess how well the generated answer addresses the question
Focuses on correctness and relevance of the response

**labeled_helpfulness_evaluator:**

Evaluates helpfulness with reference answers
Compares the generated answer against the reference answer to determine if the submission is helpful to the user
Takes into account the correct reference answer as ground truth

**empathy_evaluator:**

Evaluates empathy and emotional support
Assesses whether the response is empathetic and makes the user feel heard and understood
Checks if the response demonstrates kindness and emotional intelligence

## LangSmith Evaluation

In [46]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'advanced-neck-43' at:
https://smith.langchain.com/o/e60ce1c7-6903-49a6-9264-5c7bd472b609/datasets/51da0801-b3c4-4e6e-9266-2537fdfe2b2a/compare?selectedSessions=61164698-1139-4ca4-9534-ea1780b9cc18




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,How do Chapters 2 and 3 collectively address t...,Chapters 2 and 3 collectively address the incl...,None,Chapter 2 discusses the inclusion of clinical ...,1,1,0,6.915536,1ea3cdd8-e97f-4aaf-aa8c-2c238a71e8a7,ae682cd5-0904-42b4-8ee9-4c1228bf548f
1,Considering the disbursement timing rules for ...,"Based on the provided context, the inclusion o...",None,The inclusion of clinical work conducted outsi...,0,0,0,7.224609,2834503c-5190-4926-84a3-b565b68f8c36,ea168d25-1416-4871-b43e-311ccd8e7fae
2,How does Volume 8 explain the impact of accele...,Volume 8 explains that if clinical work meets ...,None,Volume 8 details that in clock-hour or non-ter...,0,0,0,4.076714,aa3c02d5-f9d1-4721-932e-c06c839b0ae9,e37f6436-a709-40b0-b8c8-c1812eedf248
3,How do Appendix A and Appendix B guide the dis...,"Based on the provided context, Appendix B spec...",None,Appendix B provides detailed guidance on disbu...,0,0,0,7.099299,4563b284-fd8e-4e1d-9047-1bc155853267,1fde6896-616f-4527-9f61-33fb809dc1f4
4,How does the inclusion of clinical work in sta...,The inclusion of clinical work in standard ter...,None,The inclusion of clinical work in standard ter...,1,1,0,5.133488,57af92d0-6598-41c2-a116-e4f1a85cd87d,96c97ef8-c21e-4806-b0eb-1b260dca1658
5,How do credit-hour and clock-hour programs dif...,Based on the provided context:\n\n**Minimum Ac...,None,Credit-hour programs must have an academic yea...,1,0,0,5.227769,59cd0622-5526-4d95-a68f-1287565a6bd0,23f6bd6f-0ba3-4266-975a-a1ad11de7fa6
6,Wht are the weeks of instrctional time reuired...,The weeks of instructional time required for a...,None,The context states that for credit-hour progra...,1,1,0,3.838014,20fa0241-e929-49ea-965a-5529d5e42f1f,4a382228-bc9a-4250-96e0-5b404357fd2d
7,How does completing additional hours or weeks ...,Completing additional weeks or hours in a paym...,None,Completing additional weeks of instructional t...,1,1,0,5.059131,690fb2ad-9e4b-49bb-98d9-5fa3a58b761c,b6ea1402-2988-49d8-8667-e50980308a24
8,FWS is it like a payment period or not and how...,Based on the provided context:\n\n- The paymen...,None,The payment period is applicable to all Title ...,1,1,0,3.699819,fbab6bd2-8b70-46d3-929b-cb47f0c7de79,989641e3-4940-42cb-929e-99c592b77153
9,What is Chaptr 3?,"Chapter 3 is titled ""Packaging Aid."" It explai...",None,Inclusion of Clinical Work in a Standard Term ...,0,0,0,1.727932,760adb8b-3c09-4c0d-9a15-bfaa254bc0c6,d3fc8135-9c16-48eb-a1c4-41c7a4533819


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [47]:
EMPATHY_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

You must answer the question using empathy and kindness, and make sure the user feels heard.

Context: {context}
Question: {question}
"""

empathy_rag_prompt = ChatPromptTemplate.from_template(EMPATHY_RAG_PROMPT)

In [48]:
rag_documents = docs

In [49]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

##### ✅ Answer:

Larger chunks provide more comprehensive context, reducing information fragmentation.

In [50]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

##### ✅ Answer:

text-embedding-3-large offers higher dimensional representations and better semantic capture

In [51]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Loan Data for RAG"
)

In [52]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [53]:
empathy_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | empathy_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [54]:
empathy_rag_chain.invoke({"question" : "What kinds of loans are available?"})

"Thank you for your question. Based on the information you've shared, it looks like there are a few types of federal student loans available:\n\n1. **Direct Subsidized Loans** – These are loans based on the student’s financial need, and interest is paid by the government while the student is in school at least half-time. For example, dependent first-year students can borrow up to $3,500 in subsidized loans.\n\n2. **Direct Unsubsidized Loans** – These loans are available to students regardless of financial need. The borrower is responsible for interest during all periods.\n\n3. **Direct PLUS Loans** – These loans are available to parents of dependent students (called Direct PLUS Loans for parents) or to graduate/professional students (called Direct PLUS Loans for students). They can cover the student’s cost of attendance beyond other financial aid but require the borrower to meet eligibility requirements. If a parent is unable to get a Direct PLUS Loan, the dependent student may get add

Finally, we can evaluate the new chain on the same test set!

In [55]:
evaluate(
    empathy_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "empathy_rag_chain"},
)

View the evaluation results for experiment: 'cold-nut-12' at:
https://smith.langchain.com/o/e60ce1c7-6903-49a6-9264-5c7bd472b609/datasets/51da0801-b3c4-4e6e-9266-2537fdfe2b2a/compare?selectedSessions=a17a0094-14df-4c85-b417-cb5d61709218




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,How do Chapters 2 and 3 collectively address t...,Thank you for your thoughtful question. Based ...,None,Chapter 2 discusses the inclusion of clinical ...,1,1,1,5.043275,1ea3cdd8-e97f-4aaf-aa8c-2c238a71e8a7,06e85722-0544-4433-a563-fcd9ec7787ca
1,Considering the disbursement timing rules for ...,Thank you for your thoughtful question—it’s gr...,None,The inclusion of clinical work conducted outsi...,1,1,1,7.450671,2834503c-5190-4926-84a3-b565b68f8c36,df064232-3061-4177-9ce5-62fd5d0eef96
2,How does Volume 8 explain the impact of accele...,Thank you for your thoughtful question. Based ...,None,Volume 8 details that in clock-hour or non-ter...,0,0,1,6.467466,aa3c02d5-f9d1-4721-932e-c06c839b0ae9,227db3ae-1712-4716-bdc9-d8a12810640b
3,How do Appendix A and Appendix B guide the dis...,Thank you for your thoughtful question. I can ...,None,Appendix B provides detailed guidance on disbu...,0,0,1,4.343443,4563b284-fd8e-4e1d-9047-1bc155853267,38b2ca3f-2794-4408-b0ec-d01b464aadd4
4,How does the inclusion of clinical work in sta...,Thank you for your thoughtful question. I unde...,None,The inclusion of clinical work in standard ter...,1,1,1,5.135947,57af92d0-6598-41c2-a116-e4f1a85cd87d,724449be-4836-4c48-822b-190040e34c2b
5,How do credit-hour and clock-hour programs dif...,Thank you for your thoughtful question. Based ...,None,Credit-hour programs must have an academic yea...,1,0,1,4.740575,59cd0622-5526-4d95-a68f-1287565a6bd0,e14b45f7-1498-47ea-8497-9f877a869699
6,Wht are the weeks of instrctional time reuired...,Thank you for your question! I understand that...,None,The context states that for credit-hour progra...,1,1,1,5.930687,20fa0241-e929-49ea-965a-5529d5e42f1f,5c055efb-48a1-41c0-8998-0637d6b1a507
7,How does completing additional hours or weeks ...,Thank you for your thoughtful question. Based ...,None,Completing additional weeks of instructional t...,1,0,1,4.783308,690fb2ad-9e4b-49bb-98d9-5fa3a58b761c,b0c3bb4c-3dd6-4f15-991e-8affa5482461
8,FWS is it like a payment period or not and how...,Thank you for your thoughtful question about t...,None,The payment period is applicable to all Title ...,1,1,1,4.362410,fbab6bd2-8b70-46d3-929b-cb47f0c7de79,682d0703-9211-41e0-80ec-d975d298f7b4
9,What is Chaptr 3?,Thank you for your question! Based on the cont...,None,Inclusion of Clinical Work in a Standard Term ...,0,0,1,2.354064,760adb8b-3c09-4c0d-9a15-bfaa254bc0c6,041ef0a4-370b-4e49-8417-40f24942596d


#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.

##### ✅ Answer:

**Screenshot of LangSmith Evaluation Comparison:**

![Screenshot](Screenshot2025-07-19at9.23.03 AM.png)


**Analysis of Results from Screenshot:**

Based on your LangSmith comparison screenshot, here's what the metrics are telling us:

**Observed Changes:**

Looking at the evaluation results between your two experiments (`default_chain_init` vs `empathy_rag_chain`), we see specific metric improvements in:

1. **Empathy Score**: This shows the most dramatic improvement since we explicitly added empathy instructions to the prompt. The empathy_evaluator measures whether responses make users feel heard and understood.

2. **QA Correctness**: Correctness stayed the same (although one question improved and one question regressed on this metric.) Surprising, since larger chunks + better embeddings should have provided more relevant context for answering questions accurately.

3. **Helpfulness Score**: Helpfulness (which is a measure of how similar the answer is to the reference answer) regressed by 1 point as two questions regressed (-2) nad 1 question improved (+1) w/r/t this metric. This demonstrates to me that sometimes increasing the chunk size and improving the embedding model is not the answer, as it depends on the type of data being evaluated / measured.
